---
description: Logging system for hwpapi
output-file: logging.html
title: logging
---


In [ ]:
#| default_exp logging


In [ ]:
#| hide
%load_ext autoreload
%autoreload 2


In [ ]:
#| export
import logging
import sys
from pathlib import Path
from typing import Optional, Union
import os


In [ ]:
#| export
class HwpApiLogger:
    """
    Centralized logging system for hwpapi package.
    
    This class provides a unified logging interface for the entire hwpapi package,
    with configurable logging levels, formatters, and output destinations.
    
    Attributes
    ----------
    _instance : HwpApiLogger
        Singleton instance of the logger
    _initialized : bool
        Flag to track if logger has been initialized
    logger : logging.Logger
        The underlying Python logger instance
    
    Examples
    --------
    >>> logger = HwpApiLogger.get_logger()
    >>> logger.info("Application started")
    >>> logger.debug("Debug information")
    >>> logger.error("An error occurred")
    """
    
    _instance = None
    _initialized = False
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(HwpApiLogger, cls).__new__(cls)
        return cls._instance
    
    def __init__(self):
        if not self._initialized:
            self._setup_logger()
            HwpApiLogger._initialized = True
    
    def _setup_logger(self):
        """Initialize the logger with default configuration."""
        self.logger = logging.getLogger('hwpapi')
        
        # Set default level from environment variable or INFO
        level = os.environ.get('HWPAPI_LOG_LEVEL', 'INFO').upper()
        self.logger.setLevel(getattr(logging, level, logging.INFO))
        
        # Create console handler if none exists
        if not self.logger.handlers:
            # Check if running in Jupyter notebook
            try:
                from IPython import get_ipython
                if get_ipython() is not None:
                    # Use stderr for Jupyter notebooks for better visibility
                    console_handler = logging.StreamHandler(sys.stderr)
                else:
                    console_handler = logging.StreamHandler(sys.stdout)
            except ImportError:
                # Not in Jupyter, use stdout
                console_handler = logging.StreamHandler(sys.stdout)
            
            # Create formatter
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                datefmt='%Y-%m-%d %H:%M:%S'
            )
            console_handler.setFormatter(formatter)
            
            self.logger.addHandler(console_handler)
        
        # Prevent propagation to root logger
        self.logger.propagate = False
    
    @classmethod
    def get_logger(cls, name: Optional[str] = None) -> logging.Logger:
        """
        Get a logger instance for the specified module.
        
        Parameters
        ----------
        name : str, optional
            Name of the module/component requesting the logger.
            If None, returns the root hwpapi logger.
        
        Returns
        -------
        logging.Logger
            Configured logger instance
        """
        instance = cls()
        if name:
            return logging.getLogger(f'hwpapi.{name}')
        return instance.logger
    
    def set_level(self, level: Union[str, int]):
        """
        Set the logging level for all hwpapi loggers.
        
        Parameters
        ----------
        level : str or int
            Logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
        """
        if isinstance(level, str):
            level = getattr(logging, level.upper())
        self.logger.setLevel(level)
        
        # Update all child loggers
        for handler in self.logger.handlers:
            handler.setLevel(level)
    
    def add_file_handler(self, file_path: Union[str, Path], level: Optional[Union[str, int]] = None):
        """
        Add a file handler to the logger.
        
        Parameters
        ----------
        file_path : str or Path
            Path to the log file
        level : str or int, optional
            Logging level for the file handler
        """
        file_path = Path(file_path)
        file_path.parent.mkdir(parents=True, exist_ok=True)
        
        file_handler = logging.FileHandler(file_path)
        
        if level is not None:
            if isinstance(level, str):
                level = getattr(logging, level.upper())
            file_handler.setLevel(level)
        
        # Create detailed formatter for file logs
        file_formatter = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(funcName)s:%(lineno)d - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )
        file_handler.setFormatter(file_formatter)
        
        self.logger.addHandler(file_handler)
    
    def remove_console_handler(self):
        """Remove console handler from the logger."""
        handlers_to_remove = [h for h in self.logger.handlers if isinstance(h, logging.StreamHandler)]
        for handler in handlers_to_remove:
            self.logger.removeHandler(handler)
    
    def configure(self, 
                 level: Union[str, int] = 'INFO',
                 console: bool = True,
                 file_path: Optional[Union[str, Path]] = None,
                 format_string: Optional[str] = None):
        """
        Configure the logger with custom settings.
        
        Parameters
        ----------
        level : str or int
            Logging level
        console : bool
            Whether to log to console
        file_path : str or Path, optional
            Path to log file
        format_string : str, optional
            Custom format string for log messages
        """
        # Clear existing handlers
        self.logger.handlers.clear()
        
        # Set level
        self.set_level(level)
        
        # Default format
        if format_string is None:
            format_string = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        
        formatter = logging.Formatter(format_string, datefmt='%Y-%m-%d %H:%M:%S')
        
        # Add console handler
        if console:
            # Check if running in Jupyter notebook
            try:
                from IPython import get_ipython
                if get_ipython() is not None:
                    # Use stderr for Jupyter notebooks for better visibility
                    console_handler = logging.StreamHandler(sys.stderr)
                else:
                    console_handler = logging.StreamHandler(sys.stdout)
            except ImportError:
                # Not in Jupyter, use stdout
                console_handler = logging.StreamHandler(sys.stdout)
            
            console_handler.setFormatter(formatter)
            self.logger.addHandler(console_handler)
        
        # Add file handler
        if file_path:
            self.add_file_handler(file_path, level)


In [ ]:
#| export
def get_logger(name: Optional[str] = None) -> logging.Logger:
    """
    로거 인스턴스를 가져오는 편의 함수입니다.
    
    매개변수
    ----------
    name : str, optional
        로거를 요청하는 모듈/컴포넌트의 이름
    
    반환값
    -------
    logging.Logger
        구성된 로거 인스턴스
    
    사용 예시
    --------
    >>> from hwpapi.logging import get_logger
    >>> logger = get_logger('core')
    >>> logger.info('Core module initialized')
    """
    return HwpApiLogger.get_logger(name)

def configure_logging(**kwargs):
    """
    hwpapi 로깅 시스템을 구성합니다.
    
    매개변수
    ----------
    **kwargs
        HwpApiLogger.configure()에 전달되는 구성 옵션
    
    사용 예시
    --------
    >>> from hwpapi.logging import configure_logging
    >>> configure_logging(level='DEBUG', file_path='hwpapi.log')
    """
    logger_instance = HwpApiLogger()
    logger_instance.configure(**kwargs)


In [ ]:
#| export
def setup_jupyter_logging(level='INFO', format_string=None):
    """
    더 나은 가시성을 위해 Jupyter 노트북 전용 로깅을 설정합니다.
    
    매개변수
    ----------
    level : str
        로깅 레벨 (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format_string : str, optional
        로그 메시지용 사용자 정의 형식 문자열
    
    사용 예시
    --------
    >>> from hwpapi.logging import setup_jupyter_logging
    >>> setup_jupyter_logging(level='DEBUG')
    """
    import sys
    
    # Force Jupyter-friendly configuration
    logger_instance = HwpApiLogger()
    
    # Clear existing handlers
    logger_instance.logger.handlers.clear()
    
    # Set level
    logger_instance.set_level(level)
    
    # Default format for Jupyter
    if format_string is None:
        format_string = '[%(levelname)s] %(name)s: %(message)s'
    
    formatter = logging.Formatter(format_string)
    
    # Add stderr handler for Jupyter (shows as red text which is more visible)
    console_handler = logging.StreamHandler(sys.stderr)
    console_handler.setFormatter(formatter)
    logger_instance.logger.addHandler(console_handler)
    
    # Also add stdout handler for regular output
    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setFormatter(formatter)
    logger_instance.logger.addHandler(stdout_handler)
    
    print(f"✅ Jupyter logging configured at {level} level")
    return logger_instance


In [ ]:
#| hide
# Test the logging system
print("=== Testing Basic Logging ===")
logger = get_logger('test')
logger.info('Testing logging system')
logger.debug('This is a debug message (may not be visible)')
logger.warning('This is a warning')
logger.error('This is an error')

print("\n=== Testing Jupyter-Specific Logging ===")
# Test Jupyter-specific setup
setup_jupyter_logging(level='DEBUG')
logger = get_logger('jupyter_test')
logger.debug('Debug message should now be visible in Jupyter')
logger.info('Info message in Jupyter')
logger.warning('Warning message in Jupyter')
logger.error('Error message in Jupyter')

print("\n=== Testing File Logging ===")
# Test file logging
import tempfile
with tempfile.NamedTemporaryFile(suffix='.log', delete=False) as tmp:
    configure_logging(level='INFO', file_path=tmp.name)
    logger.info('This should go to both console and file')
    print(f"Log file created at: {tmp.name}")
    
    # Read and display log file contents
    with open(tmp.name, 'r') as f:
        print("Log file contents:")
        print(f.read())


In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()
